In [1]:
import torch
import torch.nn as nn
from torch.nn import Module, Linear
from torch.nn.functional import tanh
import torch.nn.init as init
from torch.optim import AdamW, LBFGS
from torch.autograd import Variable
from torch.autograd import grad
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
from itertools import cycle

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
class ComplexTanh(Module):
    @staticmethod
    def forward(inp):
        return tanh(inp.real).type(torch.complex64) + 1j * tanh(inp.imag).type(torch.complex64)

class ComplexLinear(Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.fc_r = Linear(in_features, out_features)
        self.fc_i = Linear(in_features, out_features)

    def forward(self, inp):
        return (self.fc_r(inp.real) - self.fc_i(inp.imag)).type(torch.complex64) + 1j * (self.fc_r(inp.imag) + self.fc_i(inp.real)).type(torch.complex64)

class ComplexFeedForwardNetwork(Module):
    def __init__(self, in_features, hidden_features1, hidden_features2, hidden_features3, out_features):
        super().__init__()
        # Adjust in_features to handle two complex inputs
        self.layer1 = ComplexLinear(in_features, hidden_features1)
        self.layer2 = ComplexLinear(hidden_features1, hidden_features2)
        self.layer3 = ComplexLinear(hidden_features2, hidden_features3)
        self.activation = ComplexTanh()
        self.layer4 = ComplexLinear(hidden_features3, out_features)

    def forward(self, input1, input2):
        # Combine two complex inputs
        combined_input = torch.cat((input1, input2), dim=-1)
        x = self.layer1(combined_input)
        x = self.activation(x)
        x = self.layer2(x)
        x = self.activation(x)
        x = self.layer3(x)
        x = self.activation(x)
        x = self.layer4(x)
        return x

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ComplexFeedForwardNetwork(2, 16, 32, 16, 1).to(device)

# Create two single complex tensors from x and t
input1 = torch.tensor([3.0 + 1j * 4.0], requires_grad=True, device=device)
input2 = torch.tensor([2.0 + 1j * 5.0], requires_grad=True, device=device)

# Forward pass
output = model(input1, input2)

# Compute derivatives
output.backward(torch.ones_like(output, device=device))
print("Gradient with respect to input 1:", input1.grad)
print("Gradient with respect to input 2:", input2.grad)

Gradient with respect to input 1: tensor([-0.0817+0.0613j], device='cuda:0')
Gradient with respect to input 2: tensor([0.0804-0.1330j], device='cuda:0')


In [4]:
def analytical_u(x, t, k, omega, r):
    return torch.sqrt(torch.tensor(2)) *( 1/torch.cosh(x+t) ) * torch.exp(1j * ( -x*torch.tensor(0.5) + t*torch.tensor(0.75) ) )

def analytical_v(x, t, k, omega, r):
    result = -2 * torch.square( 1/torch.cosh(x+t) )
    return torch.complex(result, torch.zeros_like(result))

def complex_mse_loss(output, target):
    return torch.mean((output.real - target.real) ** 2 + (output.imag - target.imag) ** 2)

def compute_analytical_boundary_loss(model_u, model_v, x, t, k, omega, r):
    complex_x = torch.complex(x, torch.zeros_like(x)) 
    complex_t = torch.complex(t, torch.zeros_like(t))
    
    pred_u = model_u(complex_x,complex_t)
    pred_v =  model_v(complex_x,complex_t)

    u_val = analytical_u(x, t, k, omega, r)
    v_val = analytical_v(x, t, k, omega, r)
 
    boundary_loss_u = complex_mse_loss(pred_u, u_val)
    boundary_loss_v = complex_mse_loss(pred_v, v_val)
    
    return boundary_loss_u, boundary_loss_v

def complex_grad(outputs, inputs):
    return grad(outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True)

def compute_f_and_g(u, v, x, t):
    # Compute first derivatives
    u_t = complex_grad(u, t)[0]
    u_x = complex_grad(u, x)[0]
    v_t = complex_grad(v, t)[0]
    v_x = complex_grad(v, x)[0]
    
    u_xx = complex_grad(u_x, x)[0]
    
    u_abs_squared = u.abs()**2
    u_abs_squared_x = complex_grad(u_abs_squared, x)[0]

    # Compute the residual f and g as per the given equations
    f = 1j * u_t - u_xx - u * v
    g = v_t + u_abs_squared_x
    return f, g

In [5]:
def training(model_u, model_v, model_save_path, device, num_epochs, lr, num_samples, r, k, omega, gamma, beta, line_search_fn):
    print('Starting Training')
    optimizer_u = LBFGS(model_u.parameters(), lr=lr, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn=line_search_fn)
    optimizer_v = LBFGS(model_v.parameters(), lr=lr, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn=line_search_fn)

    factor = -2

    x_n = (torch.rand(num_samples, 1)*4 + factor ).to(device)  # x in range [-5, -3]
    t_n = (torch.rand(num_samples, 1)).to(device)   
    x_dom = (torch.rand(num_samples*30, 1)*4 + factor ).to(device)
    t_dom = torch.rand(num_samples*30, 1).to(device) 
    x_bc_x0 = (torch.zeros(num_samples, 1)*4 + factor ).to(device)
    t_bc_x0 = torch.rand(num_samples, 1).to(device)  # Uniformly distributed random values between 0 and 1
    x_bc_x1 = (torch.zeros(num_samples, 1)*4 - factor ).to(device)
    t_bc_x1 = torch.rand(num_samples, 1).to(device)  # Uniformly distributed random values between 0 and 1
    x_bc_t0 = (torch.rand(num_samples, 1)*4 + factor ).to(device)  # Uniformly distributed random values between 0 and 1
    t_bc_t0 = torch.zeros(num_samples, 1).to(device)

    complex_x_dom = torch.complex(x_dom, torch.zeros_like(x_dom)) 
    complex_t_dom = torch.complex(t_dom, torch.zeros_like(t_dom)) 
            # Set requires_grad=True 
    complex_x_dom = complex_x_dom.requires_grad_(True)
    complex_t_dom = complex_t_dom.requires_grad_(True)

    for epoch in tqdm(range(num_epochs),
                  desc='Progress:',  
                  leave=False,  
                  ncols=75,
                  mininterval=0.1,
                  bar_format='{l_bar} {bar} | {remaining}',  # Only show the bar without any counters
                  colour='blue'): 
        model_u.train()
        model_v.train()
        
        def closure_u():
            optimizer_u.zero_grad()
            optimizer_v.zero_grad()
            pred_u = model_u(complex_x_dom, complex_t_dom)
            pred_v = model_v(complex_x_dom, complex_t_dom)
            f, g = compute_f_and_g(pred_u, pred_v, complex_x_dom, complex_t_dom)
            physics_loss_u = torch.mean(torch.abs(f)**2)
            physics_loss_v = torch.mean(torch.abs(g)**2)
            print(physics_loss_u)
            boundary_loss_u_x0, boundary_loss_v_x0 = compute_analytical_boundary_loss(model_u, model_v, x_bc_x0, t_bc_x0, k, omega, r)
            boundary_loss_u_x1, boundary_loss_v_x1 = compute_analytical_boundary_loss(model_u, model_v, x_bc_x1, t_bc_x1, k, omega, r)
            boundary_loss_u_t0, boundary_loss_v_t0 = compute_analytical_boundary_loss(model_u, model_v, x_bc_t0, t_bc_t0, k, omega, r)
            loss_u = gamma*(physics_loss_u) + beta*( boundary_loss_u_x0 + boundary_loss_u_x1 + boundary_loss_u_t0)
            loss_u.backward()
            return loss_u

        def closure_v():
            optimizer_u.zero_grad()
            optimizer_v.zero_grad()
            pred_u = model_u(complex_x_dom, complex_t_dom)
            pred_v = model_v(complex_x_dom, complex_t_dom)
            f, g = compute_f_and_g(pred_u, pred_v, complex_x_dom, complex_t_dom)
            physics_loss_u = torch.mean(torch.abs(f)**2)
            physics_loss_v = torch.mean(torch.abs(g)**2)
            print(physics_loss_v)
            boundary_loss_u_x0, boundary_loss_v_x0 = compute_analytical_boundary_loss(model_u, model_v, x_bc_x0, t_bc_x0, k, omega, r)
            boundary_loss_u_x1, boundary_loss_v_x1 = compute_analytical_boundary_loss(model_u, model_v, x_bc_x1, t_bc_x1, k, omega, r)
            boundary_loss_u_t0, boundary_loss_v_t0 = compute_analytical_boundary_loss(model_u, model_v, x_bc_t0, t_bc_t0, k, omega, r)
            loss_v = gamma*(physics_loss_v) + beta*( boundary_loss_v_x0 + boundary_loss_v_x1 + boundary_loss_v_t0)
            loss_v.backward()
            return loss_v
        optimizer_u.step(closure_u)    
        optimizer_v.step(closure_v)

        if epoch % 10 == 0:
            current_loss_u = closure_u()  # Optionally recompute to print
            current_loss_v = closure_v()

            if current_loss_u.item() + current_loss_v.item() < 1e-3:
                break
            else:
                print(f' Epoch {epoch}, Loss U: {current_loss_u.item()}, Loss V: {current_loss_v.item()}') 
                model_u_filename = os.path.join(model_save_path, f'C_HIGGS_U_training_epoch_{epoch}.pth')
                torch.save(model_u.state_dict(), model_u_filename)
                model_v_filename = os.path.join(model_save_path, f'C_HIGGS_V_training_epoch_{epoch}.pth')
                torch.save(model_v.state_dict(), model_v_filename)
                plot_model_results(epoch, model_u, model_v, device, k, omega, r, sigma=1, cmap='viridis', image_save_path='results') 
            
    model_u_filename = os.path.join(model_save_path, f'C_HIGGS_U_training.pth')
    torch.save(model_u.state_dict(), model_u_filename)
    model_v_filename = os.path.join(model_save_path, f'C_HIGGS_V_training.pth')
    torch.save(model_v.state_dict(), model_v_filename)
    print('TRAINING COMPLETED')

In [6]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter

def plot_model_results(epoch, model_u, model_v, device, k, omega, r, sigma=1, cmap='viridis', image_save_path='results'):
    x = torch.linspace(-1.8, 1.8, 400)
    t = torch.linspace(0.2, 0.8, 400)
    X, T = torch.meshgrid(x, t)  # Create a 2D grid of x and t
    X_flat = X.flatten().unsqueeze(-1).to(device)
    T_flat = T.flatten().unsqueeze(-1).to(device)
    
    model_u_state = torch.load(os.path.join(model_save_path, f'C_HIGGS_U_training_epoch_{epoch}.pth'), map_location=device)
    model_u.load_state_dict(model_u_state)
    model_u.eval()

    model_v_state = torch.load(os.path.join(model_save_path, f'C_HIGGS_V_training_epoch_{epoch}.pth'), map_location=device)
    model_v.load_state_dict(model_v_state)
    model_v.eval()
    complex_X_flat = torch.complex(X_flat, torch.zeros_like(X_flat)) 
    complex_T_flat = torch.complex(T_flat, torch.zeros_like(T_flat)) 

    # Get predictions from the trained models
    with torch.no_grad():
        pred_u = model_u(complex_X_flat, complex_T_flat) 
        pred_v = model_v(complex_X_flat, complex_T_flat) 
    pred_u_r = pred_u.real 
    pred_u_i = pred_u.imag
    pred_u_r = pred_u_r.cpu().reshape(X.shape).numpy()
    pred_u_i = pred_u_i.cpu().reshape(X.shape).numpy()
    pred_v = pred_v.cpu().reshape(X.shape).numpy()

    u1_analytical = analytical_u(X_flat, T_flat, k, omega, r).cpu().reshape(X.shape).numpy()
    real_u1_analytical = u1_analytical.real
    imag_u1_analytical = u1_analytical.imag
    real_v1_analytical = analytical_v(X_flat, T_flat, k, omega, r).cpu().reshape(X.shape).numpy()

    pred_v_smooth = gaussian_filter(pred_v, sigma=sigma)

    shrink = 0.3
    aspect = 50

    # Plotting predictions
    fig = plt.figure(figsize=(24, 16))

    ax1 = fig.add_subplot(231, projection='3d')
    ax1.plot_surface(X.numpy(), T.numpy(), pred_u_r, cmap=cmap)
    ax1.set_title('Predicted Real Part of $u_1(x, t)$')
    ax1.set_xlabel('x')
    ax1.set_ylabel('t')
    ax1.set_zlabel('Real part of $u_1$')

    ax2 = fig.add_subplot(232, projection='3d')

    ax2.plot_surface(X.numpy(), T.numpy(), pred_u_i, cmap=cmap)
    ax2.set_title('Predicted Imaginary Part of $u_1(x, t)$')
    ax2.set_xlabel('x')
    ax2.set_ylabel('t')
    ax2.set_zlabel('Imag part of $u_1$')

    ax3 = fig.add_subplot(233, projection='3d')
    ax3.plot_surface(X.numpy(), T.numpy(), pred_v_smooth, cmap=cmap)
    ax3.set_title('Predicted Real Part of $v_1(x, t)$')
    ax3.set_xlabel('x')
    ax3.set_ylabel('t')
    ax3.set_zlabel('Real part of $v_1$')

    ax4 = fig.add_subplot(234, projection='3d')
    ax4.plot_surface(X.numpy(), T.numpy(), real_u1_analytical, cmap=cmap)
    ax4.set_title('Analytical Real Part of $u_1(x, t)$')
    ax4.set_xlabel('x')
    ax4.set_ylabel('t')
    ax4.set_zlabel('Real part of $u_1$')

    ax5 = fig.add_subplot(235, projection='3d')
    ax5.plot_surface(X.numpy(), T.numpy(), imag_u1_analytical, cmap=cmap)
    ax5.set_title('Analytical Imaginary Part of $u_1(x, t)$')
    ax5.set_xlabel('x')
    ax5.set_ylabel('t')
    ax5.set_zlabel('Imag part of $u_1$')

    ax6 = fig.add_subplot(236, projection='3d')
    ax6.plot_surface(X.numpy(), T.numpy(), real_v1_analytical, cmap=cmap)
    ax6.set_title('Analytical Real Part of $v_1(x, t)$')
    ax6.set_xlabel('x')
    ax6.set_ylabel('t')
    ax6.set_zlabel('Real part of $v_1$')

    plt.tight_layout()
    plt.savefig(os.path.join(image_save_path, f'chiggs_model_comparison_3d_epoch_{epoch}.png'))
    plt.close(fig)  # Close the figure to free memory


In [7]:
! rm -rf results
! rm -rf model_weights

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print("CUDA is available! Training on GPU.")
else:
    print("CUDA is not available. Training on CPU.")

model_u = ComplexFeedForwardNetwork(2, 16, 32, 16, 1).to(device)
model_v = ComplexFeedForwardNetwork(2, 16, 32, 16, 1).to(device)

print(model_u)
print(model_v)
num_epochs_lbfgs = 500  # Number of training epochs
num_samples_lbfgs = 1000 # Number of samples for training
num_epochs_sq = 36000
num_samples_sq = 1000
lr_sq = 1e-4 
lr_lbfgs = 1e-2
r = 1.1
omega = 5 
k = 0.5
gamma = 1
beta = 1
model_save_path = 'model_weights' 
os.makedirs(model_save_path, exist_ok=True)
os.makedirs('results', exist_ok=True)
losses = []
line_search_fn = "strong_wolfe"

CUDA is available! Training on GPU.
ComplexFeedForwardNetwork(
  (layer1): ComplexLinear(
    (fc_r): Linear(in_features=2, out_features=16, bias=True)
    (fc_i): Linear(in_features=2, out_features=16, bias=True)
  )
  (layer2): ComplexLinear(
    (fc_r): Linear(in_features=16, out_features=32, bias=True)
    (fc_i): Linear(in_features=16, out_features=32, bias=True)
  )
  (layer3): ComplexLinear(
    (fc_r): Linear(in_features=32, out_features=16, bias=True)
    (fc_i): Linear(in_features=32, out_features=16, bias=True)
  )
  (activation): ComplexTanh()
  (layer4): ComplexLinear(
    (fc_r): Linear(in_features=16, out_features=1, bias=True)
    (fc_i): Linear(in_features=16, out_features=1, bias=True)
  )
)
ComplexFeedForwardNetwork(
  (layer1): ComplexLinear(
    (fc_r): Linear(in_features=2, out_features=16, bias=True)
    (fc_i): Linear(in_features=2, out_features=16, bias=True)
  )
  (layer2): ComplexLinear(
    (fc_r): Linear(in_features=16, out_features=32, bias=True)
    (fc_i

In [9]:
training(model_u, model_v, model_save_path, device, num_epochs_lbfgs, lr_lbfgs, num_samples_lbfgs, r, k, omega, gamma, beta, line_search_fn)

Starting Training


Progress::   0%|                                                        | ?

tensor(0.1290, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1288, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1277, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1219, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0849, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0848, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0837, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0910, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0910, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0910, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0913, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0913, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0912, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0907, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0907, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0910, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1034, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1034, device='cuda:0',

/mnt/research/Physics-Informed-Neural-Networks-for-Quantum-Dynamics/.mamba/envs/pytorch/lib/python3.12/site-packages/matplotlib/cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)
Progress::   0%|                                                  | 8:48:54

tensor(0.6468, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6453, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6322, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.5126, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.5063, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.4501, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1036, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1037, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1043, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1108, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1107, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1104, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1077, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1078, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1085, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1163, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1164, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1169, device='cuda:0',

...

Progress::  80%| ██████████████████████████████████████▎          | 1:43:32

tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0',

Progress::  80%| ██████████████████████████████████████▍          | 1:41:35

tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0001, device='cuda:0',

TRAINING COMPLETED
